## Install Neccesssary Libraries

In [1]:
pip install -r "../requirements.txt"

## Establish connection to API

In [2]:
from amadeus import Client, ResponseError
amadeus = Client(
    client_id='vsNE2cVrLzYVgWYcF1npWpv3HYA5ICs7',
    client_secret='rasieAFzfhNCXA0D'
)

## Connect to CSV file

In [1]:
import pandas as pd 
pd.set_option('display.max_row', 1000)
df = pd.read_csv("../raw_data/sfo/listings-1.csv") 

In [9]:
neighbourhood_unique = df.neighbourhood_cleansed.unique()
#neighbourhood_unique

## Group and Sort listings by neighborhood

In [10]:
# number of listings used to get the avg safe index score
k = 5

df_grouped = df.groupby('neighbourhood_cleansed')

# group the listings into neighborhoods
top_n_rows = df_grouped.head(k)

# sort the listings after grouping for better visulaization
top_n_rows_sorted = top_n_rows.sort_values(by=['neighbourhood_cleansed'], ignore_index=True)

# check if all records have more than k listing
records_per_neighborhood = top_n_rows_sorted.groupby(['neighbourhood_cleansed']).size().reset_index(name='counts')

# filter neighborhoods that have less than to predeinfe values, k
filtered_neighborhoods = records_per_neighborhood[records_per_neighborhood['counts'] >= k]

# get records that have more than k listing
final_records = top_n_rows_sorted[top_n_rows_sorted['neighbourhood_cleansed'].isin(filtered_neighborhoods.neighbourhood_cleansed.unique())]

In [12]:
final_records.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,110825,https://www.airbnb.com/rooms/110825,20200815122856,2020-08-15,Charming Cottage with Bay View,"<b>The space</b><br />Charming, beautiful cozy...",NaN,https://a0.muscache.com/pictures/922366/9f47e1...,568933,https://www.airbnb.com/users/show/568933,...,10.0,9.0,10.0,NaN,f,1,1,0,0,0.08
1,505530,https://www.airbnb.com/rooms/505530,20200815122856,2020-08-15,Bernal Hill Farm House -with views,"Welcome home! This eclectic farm house, joins ...",Close to everything but with the feeling that ...,https://a0.muscache.com/pictures/df4c2085-c398...,698337,https://www.airbnb.com/users/show/698337,...,10.0,9.0,9.0,City registration pending,f,4,2,2,0,0.08
2,654279,https://www.airbnb.com/rooms/654279,20200815122856,2020-08-16,Quiet Room in Our Friendly Home,Private room for up to two guests in our home ...,Our neighborhood is one of the sunniest in the...,https://a0.muscache.com/pictures/a9fc113d-a62a...,3289160,https://www.airbnb.com/users/show/3289160,...,10.0,10.0,10.0,STR-0004235\n\n(Formerly STR-0000161),f,1,0,1,0,3.26
3,522273,https://www.airbnb.com/rooms/522273,20200815122856,2020-08-16,Amazing Private Room w/View in Beautiful SF Flat,Newly redecorated large private room w/ amazin...,This is a quiet tight knit neighborhood. Some...,https://a0.muscache.com/pictures/9dcd42c8-9db6...,2568946,https://www.airbnb.com/users/show/2568946,...,10.0,9.0,10.0,NaN,f,1,0,1,0,0.28
4,631735,https://www.airbnb.com/rooms/631735,20200815122856,2020-08-16,Comfy Room in SF Victorian house,The room we offer is in our Queen Ann Victoria...,"Personally, I enjoy the area for its sunny wea...",https://a0.muscache.com/pictures/9109955/5852f...,3141708,https://www.airbnb.com/users/show/3141708,...,9.0,7.0,9.0,NaN,f,1,0,1,0,0.30


## Get Response

In [ ]:
'''
# Flight Most Booked Destinations
amadeus.travel.analytics.air_traffic.booked.get(originCityCode='MAD', period='2017-08')

# Flight Most Traveled Destinations
amadeus.travel.analytics.air_traffic.traveled.get(originCityCode='MAD', period='2017-01')

# Flight Busiest Travel Period
amadeus.travel.analytics.air_traffic.busiest_period.get(cityCode='MAD', period='2017', direction='ARRIVING')

# Safe Place
# How safe is Barcelona? (based a geo location and a radius)
amadeus.safety.safety_rated_locations.get(latitude=41.397158, longitude=2.160873)
# How safe is Barcelona? (based on a square)
amadeus.safety.safety_rated_locations.by_square.get(north=41.397158, west=2.160873,
                                                    south=41.394582, east=2.177181)
# What is the safety information of a location based on it's Id?
amadeus.safety.safety_rated_location('Q930400801').get()
'''

In [13]:
safeCategory = ['overall', 'lgbtq', 'medical', 'physicalHarm', 'politicalFreedom', 'theft', 'women']

# for each neighborhood creating empty dict with all the safe category
safeIndexScoring = { j: {k:0 for k in safeCategory} for j in filtered_neighborhoods.neighbourhood_cleansed.unique()}

### Api calling

In [14]:
for index, row in final_records.iterrows():
    # retrieve data from Api
    response = amadeus.safety.safety_rated_locations.get(latitude=row['latitude'], longitude=row['longitude'])
    
    # add the each safe category score to the respective category in dictionary 
    for cat in safeCategory:
        safeIndexScoring[row['neighbourhood_cleansed']][cat] += response.data[0]['safetyScores'][cat]
        

In [16]:
# now that we have the total safety score we need to avg them to get the avg safety score
safeIndexScoring_copy= safeIndexScoring.copy()

denominator = filtered_neighborhoods.values.tolist()
tmp = {}

for area,listingCount in denominator:
    tmp[area] = {cat: round(total/listingCount,2) for cat, total in safeIndexScoring_copy[area].items()}

### Store result into Pickle file

In [22]:
output = pd.DataFrame.from_records(
            [
                (level1, level2, leaf)
                for level1, level2_dict in tmp.items()
                for level2, leaf in level2_dict.items()
            ],
            columns=['Area', 'Category', 'Score']
        )
output.to_pickle("./safeIndex.pkl")

### Read Pickle File

In [30]:
unpickled_df = pd.read_pickle("safeIndex.pkl")
unpickled_df

,Area,Category,Score
0,Bayview,overall,51.8
1,Bayview,lgbtq,46.0
2,Bayview,medical,76.6
3,Bayview,physicalHarm,46.4
4,Bayview,politicalFreedom,31.4
5,Bayview,theft,68.8
6,Bayview,women,38.6
7,Bernal Heights,overall,51.0
8,Bernal Heights,lgbtq,46.0
9,Bernal Heights,medical,75.0


## Normalize 

In [2]:
from sklearn.preprocessing import MinMaxScaler

In [31]:
x = unpickled_df[['Score']].values.astype(float)
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
df_normalized = pd.DataFrame(x_scaled, columns= ['Normalized_score'])
unpickled_df["Normalized_score"] = df_normalized['Normalized_score']
unpickled_df.drop(columns = 'Score', inplace = True)

In [32]:
unpickled_df

,Area,Category,Normalized_score
0,Bayview,overall,0.581538
1,Bayview,lgbtq,0.492308
2,Bayview,medical,0.963077
3,Bayview,physicalHarm,0.498462
4,Bayview,politicalFreedom,0.267692
5,Bayview,theft,0.843077
6,Bayview,women,0.378462
7,Bernal Heights,overall,0.569231
8,Bernal Heights,lgbtq,0.492308
9,Bernal Heights,medical,0.938462


In [33]:
unpickled_df.to_pickle("./safeIndex_normalized.pkl")